In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()


pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210611-1511-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [2]:
run_hash = "ae281191"
ht=hl.read_table(f'{lustre_dir}/variant_qc/models/{run_hash}_rf_result_FINAL_for_RANKING.ht')
run_hash = "94f5cc00"
ht2=hl.read_table(f'{lustre_dir}/variant_qc/models/94f5cc00_rf_result_transmitted_singletons_final.ht')

In [3]:
ht.count()


7515314

In [5]:
ht.variant_untransmitted_singletons.summarize()

KeyboardInterrupt: 

In [6]:
ht.variant_transmitted_singletons.summarize()

Non-missing,65286 (0.87%)
Missing,7450028 (99.13%)
Minimum,0
Maximum,4
Mean,0.78
Std Dev,0.47


In [11]:
ht2.describe()

----------------------------------------
Global fields:
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'a_index': int32 
    'was_split': bool 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<float64> 
    'AS_FS': array<float64> 
    'AS_InbreedingCoeff': array<float64> 
    'AS_MQ': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'BaseQRankSum': float64 
    'DB': bool 
    'DP': int32 
    'DS': bool 
    'END': int32 
    'ExcessHet': float64 
    'FS': float64 
    'InbreedingCoeff': float32 
    '

In [13]:
ht2.locus.summarize()

Non-missing,28008043 (100.00%)
Missing,0
Contig Counts,"{'chr11': 1632197, 'chr5': 1342615, 'chr22': 696147, 'chr8': 1104562, 'chr19': 1662084, 'chr1': 2759565, 'chr15': 1077895, 'chr12': 1571754, 'chr18': 563132, 'chr20': 720998, 'chr2': 2099677, 'chr13': 596180, 'chr7': 1431748, 'chr14': 991416, 'chr3': 1653803, 'chr17': 1555676, 'chr4': 1200680, 'chr6': 1360406, 'chr9': 1159584, 'chr10': 1147841, 'chr21': 354289, 'chr16': 1325794}"


In [23]:
ht3=ht
ht3=ht3.drop(ht3.variant_untransmitted_singletons)
ht3=ht.annotate(variant_untransmitted_singletons=ht2[ht.key].variant_untransmitted_singletons)

In [24]:
ht3.variant_untransmitted_singletons.summarize()

Non-missing,65286 (0.87%)
Missing,7450028 (99.13%)
Minimum,0
Maximum,0
Mean,0.00
Std Dev,0.00


In [25]:
ht.count()

7515314

In [26]:
ht2.count()

28008043

In [27]:
ht2.variant_untransmitted_singletons.summarize()

Non-missing,550768 (1.97%)
Missing,27457275 (98.03%)
Minimum,0
Maximum,4
Mean,0.11
Std Dev,0.32


In [31]:
ht2.variant_untransmitted_singletons.show()
#ht4=!hl.missing(ht2)

,,
locus,alleles,variant_untransmitted_singletons
locus<GRCh38>,array<str>,int64
chr1:12938,"[""GCAAA"",""G""]",NA
chr1:13024,"[""G"",""A""]",NA
chr1:13087,"[""A"",""G""]",NA
chr1:13116,"[""T"",""C""]",NA
chr1:13130,"[""C"",""T""]",NA
chr1:13151,"[""G"",""C""]",NA
chr1:13164,"[""G"",""A""]",NA
chr1:13176,"[""G"",""T""]",NA


In [30]:
ht4.describe()

AttributeError: 'SList' object has no attribute 'describe'